In [11]:
%pip install selenium webdriver-manager requests

Note: you may need to restart the kernel to use updated packages.


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import json

def open_browser(url):
    options = Options()
    # options.add_argument("--headless")  # Chạy chế độ không hiển thị (bắt buộc trên Colab)
    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")
    # options.binary_location = "/usr/bin/google-chrome"  # Đường dẫn Chrome trên Colab

    # service = Service(ChromeDriverManager().install())  # Tự động tải ChromeDriver
    # driver = webdriver.Chrome(service=service, options=options)

    # Cấu hình Chrome headless (không hiện cửa sổ)
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')

    # Tạo driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    print(f"🌐 Đang mở URL: {url}")
    driver.get(url)
    time.sleep(5) #đợi trước 5 giây
    #tiếp tục đợi nếu page chưa load xong
    while driver.page_source.find("Just a moment..") != -1:
        print("⏳ Chờ đợi trang web tải...")
        driver.get(url)
        time.sleep(5)
    html_content_gbk = ""

    try:
        # content = WebDriverWait(driver, 10).until(
        #     EC.visibility_of_element_located((By.CSS_SELECTOR, "div.txtnav, div.content"))
        # ).text
        # print(f"✅ Lấy nội dung trang web thành công ({len(content)} ký tự)")

        # print(content)

        html_content_gbk = driver.page_source.encode("utf-8").decode("gbk", errors="ignore")
        # print(html_content_gbk)
        
    except Exception as e:
        # print(driver.page_source)
        print("❌ Không tìm thấy nội dung cần dịch!")
        print(f"❌ Lỗi khi lấy nội dung trang web: {e}")
        html_content_gbk = ""

    driver.quit()
    return html_content_gbk

def get_book_info(html: str):
    match = re.search(r'var\s+bookinfo\s*=\s*(\{.*?\});', html, re.DOTALL)
    if not match:
        return None

    bookinfo_text = match.group(1)

    # Chuyển JS object → JSON-compatible string
    bookinfo_text = re.sub(r'(\w+):', r'"\1":', bookinfo_text)  # key: → "key":
    bookinfo_text = re.sub(r',\s*}', '}', bookinfo_text)        # bỏ dấu , cuối object
    bookinfo_text = re.sub(r',\s*]', ']', bookinfo_text)        # bỏ dấu , cuối array

    try:
        return json.loads(bookinfo_text)
    except json.JSONDecodeError as e:
        print("JSON decode error:", e)
        return None


html_content_gbk = open_browser("https://www.69shuba.com/book/89532.htm")

get_book_info(html_content_gbk)

JSON decode error: Expecting value: line 3 column 24 (char 54)
None


In [ ]:
import requests

data = {
            "start": 100,
            "objectid": "56909",
            "objecttype": "69shu"
        }
response = requests.post("http://14.225.254.182/io/comment/webComments",data=data)
print(response.content.decode("utf-8"))

<center><br>Không có bình luận</center><div id='cmtwd' data-start='0'></div>
